In [0]:
import requests
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.functions import col


In [0]:
airports_england_data= spark.table("dataexpert.kanak_uk_airport_geocode")

display(airports_england_data)

icao,iata,airport,city,country,latitude_decimal_degrees,longitude_decimal_degrees
EGAB,ENK,ST ANGELO,ENNISKILLEN,ENGLAND,54.399,-7.652
EGBB,BHX,BIRMINGHAM,BIRMINGHAM,ENGLAND,52.454,-1.748
EGBE,CVT,COVENTRY,COVENTRY,ENGLAND,52.37,-1.48
EGBJ,GLO,GLOUCESTERSHIRE,GOLOUCHESTERSHIRE,ENGLAND,51.894,-2.167
EGCC,MAN,MANCHESTER,MANCHESTER,ENGLAND,53.354,-2.275
EGDG,NQY,ST MAWGAN,NEWQUAI,ENGLAND,50.441,-4.995
EGFH,SWS,SWANSEA,SWANSEA,ENGLAND,51.605,-4.068
EGGD,BRS,BRISTOL,BRISTOL,ENGLAND,51.383,-2.719
EGGP,LPL,LIVERPOOL,LIVERPOOL,ENGLAND,53.334,-2.85
EGGW,LTN,LUTON,LONDON,ENGLAND,51.874,-0.368


In [0]:
uk_airports_iata = [
    row["iata"] for row in airports_england_data
    .filter(col("iata").isNotNull())
    .select("iata")
    .distinct()
    .collect()
]



	image.png
  Rank	Airport Name	IATA Code
1️⃣	London Heathrow Airport	LHR
2️⃣	London Gatwick Airport	LGW
3️⃣	Manchester Airport	MAN
4️⃣	London Stansted Airport	STN
5️⃣	London Luton Airport	LTN

In [0]:
%python
import requests
from pyspark.sql.functions import current_timestamp

API_KEY = "Your api key"

BASE_URL = "http://api.aviationstack.com/v1/flights"

def flatten_flight(flight):
    """
    Flattens nested dictionaries in the API response.
    """
    flattened = {
        "flight_date": flight.get("flight_date", ""),
        "flight_status": flight.get("flight_status", "")
    }

    for key, value in flight.items():
        if key in ["departure", "arrival", "airline", "flight", "aircraft", "live"] and isinstance(value, dict):
            for inner_key, inner_val in value.items():
                if inner_key == "codeshared" and isinstance(inner_val, dict):
                    for cs_key, cs_val in inner_val.items():
                        flattened[f"flight_codeshared_{cs_key}"] = str(cs_val) if cs_val is not None else ""
                else:
                    flattened[f"{key}_{inner_key}"] = str(inner_val) if inner_val is not None else ""
        elif key not in ["flight_date", "flight_status"]:
            flattened[key] = str(value) if value is not None else ""

    return flattened

# Separate lists for departure and arrival flights
departure_flights_data = []
arrival_flights_data = []

for airport in uk_airports_iata:
    # Fetch departure flights
    dep_params = {"access_key": API_KEY, "dep_iata": airport}
    dep_response = requests.get(BASE_URL, params=dep_params)

    if dep_response.status_code == 200:
        dep_data = dep_response.json().get("data", [])
        if dep_data:
            flattened_dep_data = [flatten_flight(flight) for flight in dep_data]
            departure_flights_data.extend(flattened_dep_data)
            print(f" departing flights from {airport}.")
        else:
            print(f"No departing flight data for {airport}.")
    else:
        print(f"Error fetching departing flights for {airport}: {dep_response.status_code}")

    # Fetch arrival flights
    arr_params = {"access_key": API_KEY, "arr_iata": airport}
    arr_response = requests.get(BASE_URL, params=arr_params)

    if arr_response.status_code == 200:
        arr_data = arr_response.json().get("data", [])
        if arr_data:
            flattened_arr_data = [flatten_flight(flight) for flight in arr_data]
            arrival_flights_data.extend(flattened_arr_data)
            print(f" arriving flights to {airport}.")
        else:
            print(f"No arriving flight data for {airport}.")
    else:
        print(f"Error fetching arriving flights for {airport}: {arr_response.status_code}")

# Convert to PySpark DataFrames & Save

# Save Departing Flights
if departure_flights_data:
    dep_flights_df = spark.createDataFrame(departure_flights_data)
    dep_flights_df = dep_flights_df.withColumn("uploaded_timestamp", current_timestamp())

    dep_flights_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("dataexpert.kanak_uk_departing_flights")
    print("Table 'uk_departing_flights' successfully created!")
else:
    print("No departing flight data available.")

# Save Arriving Flights
if arrival_flights_data:
    arr_flights_df = spark.createDataFrame(arrival_flights_data)
    arr_flights_df = arr_flights_df.withColumn("uploaded_timestamp", current_timestamp())

    arr_flights_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("dataexpert.kanak_uk_arriving_flights")
    print("Table 'uk_arriving_flights' successfully created!")
else:
    print("No arriving flight data available.")

 departing flights from NWI.
 arriving flights to NWI.
 departing flights from BLK.
 arriving flights to BLK.
 departing flights from BQH.
 arriving flights to BQH.
 departing flights from CEG.
 arriving flights to CEG.
 departing flights from SEN.
 arriving flights to SEN.
 departing flights from JER.
 arriving flights to JER.
 departing flights from BHX.
 arriving flights to BHX.
No departing flight data for QCY.
No arriving flight data for QCY.
 departing flights from LTN.
 arriving flights to LTN.
 departing flights from BZZ.
 arriving flights to BZZ.
 departing flights from LPL.
 arriving flights to LPL.
 departing flights from OXF.
 arriving flights to OXF.
 departing flights from SOU.
 arriving flights to SOU.
 departing flights from EMA.
 arriving flights to EMA.
No departing flight data for BWF.
No arriving flight data for BWF.
No departing flight data for CVT.
No arriving flight data for CVT.
No departing flight data for PLH.
No arriving flight data for PLH.
 departing flight

In [0]:
display(arr_flights_df)

aircraft,airline_iata,airline_icao,airline_name,arrival_actual,arrival_actual_runway,arrival_airport,arrival_baggage,arrival_delay,arrival_estimated,arrival_estimated_runway,arrival_gate,arrival_iata,arrival_icao,arrival_scheduled,arrival_terminal,arrival_timezone,departure_actual,departure_actual_runway,departure_airport,departure_delay,departure_estimated,departure_estimated_runway,departure_gate,departure_iata,departure_icao,departure_scheduled,departure_terminal,departure_timezone,flight_codeshared_airline_iata,flight_codeshared_airline_icao,flight_codeshared_airline_name,flight_codeshared_flight_iata,flight_codeshared_flight_icao,flight_codeshared_flight_number,flight_date,flight_iata,flight_icao,flight_number,flight_status,live,flight_codeshared,aircraft_iata,aircraft_icao,aircraft_icao24,aircraft_registration,live_altitude,live_direction,live_is_ground,live_latitude,live_longitude,live_speed_horizontal,live_speed_vertical,live_updated,uploaded_timestamp
,AF,AFR,Air France,2025-03-31T16:47:00+00:00,2025-03-31T16:47:00+00:00,Norwich International Airport,,,2025-03-31T16:55:00+00:00,2025-03-31T16:47:00+00:00,,NWI,EGSH,2025-03-31T16:50:00+00:00,,Europe/London,2025-03-31T17:18:00+00:00,2025-03-31T17:18:00+00:00,Schiphol,18,2025-03-31T17:00:00+00:00,2025-03-31T17:18:00+00:00,D6,AMS,EHAM,2025-03-31T17:00:00+00:00,2,Europe/Amsterdam,kl,klm,klm,kl1061,klm1061,1061,2025-03-31,AF3200,AFR3200,3200,landed,,null,null,null,null,null,null,null,null,null,null,null,null,null,2025-03-31T18:37:34.386Z
,VS,VIR,Virgin Atlantic,2025-03-31T08:59:00+00:00,2025-03-31T08:59:00+00:00,Norwich International Airport,,,2025-03-31T09:02:00+00:00,2025-03-31T08:59:00+00:00,,NWI,EGSH,2025-03-31T09:05:00+00:00,,Europe/London,2025-03-31T09:27:00+00:00,2025-03-31T09:27:00+00:00,Schiphol,13,2025-03-31T09:15:00+00:00,2025-03-31T09:27:00+00:00,D6,AMS,EHAM,2025-03-31T09:15:00+00:00,2,Europe/Amsterdam,kl,klm,klm,kl1055,klm1055,1055,2025-03-31,VS6921,VIR6921,6921,landed,,null,null,null,null,null,null,null,null,null,null,null,null,null,2025-03-31T18:37:34.386Z
,T3,EZE,Eastern Airways,,,Norwich International Airport,,,,,,NWI,EGSH,2025-03-31T09:05:00+00:00,,Europe/London,,,Schiphol,14,2025-03-31T09:15:00+00:00,,D6,AMS,EHAM,2025-03-31T09:15:00+00:00,2,Europe/Amsterdam,null,null,null,null,null,null,2025-03-31,T31055,EZE1055,1055,scheduled,,,null,null,null,null,null,null,null,null,null,null,null,null,2025-03-31T18:37:34.386Z
null,UX,AEA,Air Europa,2025-03-31T11:42:00+00:00,2025-03-31T11:42:00+00:00,Norwich International Airport,,20,2025-03-31T11:45:00+00:00,2025-03-31T11:42:00+00:00,,NWI,EGSH,2025-03-31T11:22:00+00:00,,Europe/London,2025-03-31T10:24:00+00:00,2025-03-31T10:24:00+00:00,Son Sant Joan Airport,25,2025-03-31T10:00:00+00:00,2025-03-31T10:24:00+00:00,,PMI,LEPA,2025-03-31T10:00:00+00:00,,Europe/Madrid,null,null,null,null,null,null,2025-03-31,UX6,AEA6,6,landed,,,B738,B738,34538E,EC-MPG,null,null,null,null,null,null,null,null,2025-03-31T18:37:34.386Z
,,HMB,CHC Helicopter,2025-03-31T13:06:00+00:00,2025-03-31T13:06:00+00:00,Norwich International Airport,,21,2025-03-31T13:12:00+00:00,2025-03-31T13:06:00+00:00,,NWI,EGSH,2025-03-31T12:45:00+00:00,,Europe/London,2025-03-31T11:00:00+00:00,2025-03-31T11:00:00+00:00,Dyce,31,2025-03-31T10:30:00+00:00,2025-03-31T11:00:00+00:00,,ABZ,EGPD,2025-03-31T10:30:00+00:00,,Europe/London,null,null,null,null,null,null,2025-03-31,,HMB,,landed,,,null,null,null,null,null,null,null,null,null,null,null,null,2025-03-31T18:37:34.386Z
null,KL,KLM,KLM,2025-03-31T13:41:00+00:00,2025-03-31T13:41:00+00:00,Norwich International Airport,,6,,2025-03-31T13:41:00+00:00,,NWI,EGSH,2025-03-31T13:35:00+00:00,,Europe/London,2025-03-31T14:06:00+00:00,2025-03-31T14:06:00+00:00,Schiphol,22,2025-03-31T13:45:00+00:00,2025-03-31T14:06:00+00:00,D6,AMS,EHAM,2025-03-31T13:45:00+00:00,2,Europe/Amsterdam,null,null,null,null,null,null,2025-03-31,KL1057,KLM1057,1057,landed,,,E190,E190,48507F,PH-EXA,null,null,null,null,null,null,null,null,2025-03-31T18:37:34.386Z
,AF,AFR,